In [1]:
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger('anonymigraph').setLevel(logging.INFO)

In [2]:
from anonymigraph.anonymization import (
    KDegreeAnonymizer,
    RandomEdgeAddDelAnonymizer,
    ConfigurationModelAnonymizer,
    NestModelAnonymizer,
    PygmalionModelAnonymizer,
    PrivateColorAnonymizer,
    SoftColorAnonymizer,
)

import networkx as nx

In [ ]:
from scipy.sparse.linalg import eigs
import numpy as np

n = 400
tau1 = 3
tau2 = 1.5
mu = 0.1
G = nx.LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=6, min_community=100, seed=10
)
G.remove_edges_from(nx.selfloop_edges(G))
assert len(list(nx.selfloop_edges(G))) == 0


eigenvalues, _ = eigs(nx.adjacency_matrix(G).astype(np.float64), k=1, which='LM')
max_alpha = 1 / np.abs(eigenvalues).max()

alpha=0.5*max_alpha
beta=1
print(G)
print("Alpha:", alpha)

# Compile Numba code
PrivateColorAnonymizer(w=1e1, alpha=alpha, is_eager=True, use_optimal1d=False).anonymize(G, random_seed=42)

In [ ]:
import time


results = {}
method_timeouts = {"PrivateColor(w=1e-3, is_eager=False)":10_000, "SoftColorAnonymizer(w=1e-3)":10_000}

seed = 42
for n in [100, 1_000, 10_000]: #10_000]:
    G = nx.barabasi_albert_graph(n=n, m=5, seed=seed)
    eigenvalues, _ = eigs(nx.adjacency_matrix(G).astype(np.float64), k=1, which='LM')
    max_alpha = 1 / np.abs(eigenvalues).max()
    alpha = 0.5 * max_alpha
    beta = 1

    methods = {}



    methods["Configuration Model"] = ConfigurationModelAnonymizer()

    methods["NestModel(depth=1)"] = NestModelAnonymizer(depth=1, r=10)
    methods["NestModel(depth=2)"] = NestModelAnonymizer(depth=2, r=10)
    methods["NestModel(depth=3)"] = NestModelAnonymizer(depth=3, r=10)

    methods[f"{16}% Random Edge Add/Del"] = RandomEdgeAddDelAnonymizer(m = int(16/100*G.number_of_edges()))
    methods[f"{8}% Random Edge Add/Del"] = RandomEdgeAddDelAnonymizer(m = int(8/100*G.number_of_edges()))
    methods[f"{4}% Random Edge Add/Del"] = RandomEdgeAddDelAnonymizer(m = int(4/100*G.number_of_edges()))

    methods[f"{64}-Degree Anonymity"] = KDegreeAnonymizer(k=64)
    methods[f"{16}-Degree Anonymity"] = KDegreeAnonymizer(k=16)
    methods[f"{4}-Degree Anonymity"] = KDegreeAnonymizer(k=4)

    methods["Pygmalion(eps = 1)"] = PygmalionModelAnonymizer(eps = 1)
    methods["Pygmalion(eps = 100)"] = PygmalionModelAnonymizer(eps = 100)
    methods["Pygmalion(eps = ∞)"] = PygmalionModelAnonymizer(eps = 100_000_000)

    methods["PrivateColor(w=1e1, is_eager=True)"] = PrivateColorAnonymizer(w=1e1, alpha=alpha, is_eager=True, use_optimal1d=False)

    methods["PrivateColor(w=1e-3, is_eager=True)"] = PrivateColorAnonymizer(w=1e-3, alpha=alpha, is_eager=True, use_optimal1d=False)
    methods["PrivateColor(w=1e-3, use_optimal1d=True)"] = PrivateColorAnonymizer(w=1e-3, alpha=alpha, is_eager=False, use_optimal1d=True)
    methods["PrivateColor(w=1e-3, is_eager=False)"] = PrivateColorAnonymizer(w=1e-3, alpha=alpha, is_eager=False, use_optimal1d=False)
    methods["SoftColorAnonymizer(w=1e-3)"] = SoftColorAnonymizer(
        k_max=40, w=1e-3,use_katz_utility = True,use_entropy_reg = True,
        alpha=alpha, beta=beta, eps_utility = 1e-9, eps_privacy = 1e-9,
        lr=0.1, patience=20, threshold=1e-3, initial_lam=1e-7, factor=1.1, device='cpu'
    )


    results[n] = {}
    for method_name, method in methods.items():
        if method_timeouts.get(method_name, 1e10) <= n:
            print(f"Skipping {method_name}: Due to prior recored timeout")
            results[n][method_name] = None
            continue

        start_time = time.time()
        _ = method.anonymize(G, random_seed=seed)
        results[n][method_name] = time.time() - start_time


In [ ]:

for n in results.keys():
    print(f"\nResults for n = {n}:")
    for method_name, time_taken in results[n].items():
        if time_taken is not None:
            print(f"{method_name}: {time_taken:.2f} seconds")
        else:
            print(f"{method_name}: Timeout or Error")
    print("-" * 40)

In [ ]:
import pandas as pd

rows = []
for n in results.keys():
    for method_name, time_taken in results[n].items():
        rows.append({'n': n, 'method': method_name, 'time': time_taken})

df = pd.DataFrame(rows)
df_pivot = df.pivot(index='n', columns='method', values='time')
latex = df_pivot.transpose().to_latex(index=True,
                         caption="Descriptive statistics for the normalized suboptimality gaps for the proposed algorithms across various graphs and hyperparameter configurations.",
                         label="tab:exp1_descriptive",
                         bold_rows=True,
                         column_format="lcccc",
                         float_format="%.2f"
)
print(latex)